In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

image1=cv.imread(r'')
image2=cv.imread(r'')

#turn into gray scale
gray_image1=cv.cvtColor(image1,cv.COLOR_BGR2GRAY)
gray_image2=cv.cvtColor(image2,cv.COLOR_BGR2GRAY)

#Apply GaussianBlur
blurred_image1 = cv.GaussianBlur(gray_image1, (7, 7), 0)
blurred_image2 = cv.GaussianBlur(gray_image2, (7, 7), 0)

# Canny for edge detection
edges1 = cv.Canny(blurred_image1, 50, 150)
edges2 = cv.Canny(blurred_image2, 50, 150)

plt.imshow(gray_image1)
plt.title('image1')
plt.show()
plt.imshow(gray_image2)
plt.title('image2')
plt.show()

plt.subplot(1, 2, 2)
plt.imshow(edges1, cmap='gray')
plt.title('Canny Edge Detection')

plt.show()
plt.subplot(1, 2, 2)
plt.imshow(edges2, cmap='gray')
plt.title('Canny Edge Detection')
 
plt.show()

#Adaptive thresholding for segmenting the image
adaptive_thresh_image1 = cv.adaptiveThreshold(edges1, 255, 
                                              cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                              cv.THRESH_BINARY, 11, 2)
adaptive_thresh_image2 = cv.adaptiveThreshold(edges2, 255, 
                                              cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                              cv.THRESH_BINARY, 11, 2)

plt.imshow(adaptive_thresh_image1, cmap='gray')
plt.title('Adaptive Thresholded Image1')
plt.show()
plt.imshow(adaptive_thresh_image2, cmap='gray')
plt.title('Adaptive Thresholded Image2')
plt.show()


#SIFT-FLANN

sift=cv.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(adaptive_thresh_image1, None)
keypoints2, descriptors2 = sift.detectAndCompute(adaptive_thresh_image2, None)

print(f"Number of keypoints in image1: {len(keypoints1)}")
print(f"Number of keypoints in image2: {len(keypoints2)}")

FLANN_INDEX_KDTREE=1
index_params=dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params=dict(checks=50)

flann=cv.FlannBasedMatcher(index_params,search_params)

matches = flann.knnMatch(descriptors1, descriptors2, k=2)
good_matches = []
for m, n in matches:
    if m.distance<0.75* n.distance:
        good_matches.append(m)
matched_image = cv.drawMatches(adaptive_thresh_image1, keypoints1, adaptive_thresh_image2, keypoints2, good_matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.imshow(matched_image)
plt.title('FLANN Keypoint Matches')
plt.show()
num_good_matches = len(good_matches)
print(f"Number of good matches: {num_good_matches}")